In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import pickle
import re

## Getting Category Products Details

In [ ]:
category = "electromenager"
products = []
num_pages = 100
for page in range(1,num_pages):
    try:
        print("current page:", page)
        url = f"https://www.jumia.dz/{category}/?page={page}#catalog-listing"
        req = requests.get(url)
        s = BeautifulSoup(req.content, "html.parser")
        script_tags = s.find_all("script")
        
        for script_tag in script_tags:
            sc = script_tag.string
            if "window.__STORE__=" in sc:
                # print(sc)
                data = sc.split('__=')[1][:-1]
                data = json.loads(data)
                data = data["products"]
                # print(data)
                print("products: ", len(data))
                for product in data:
                    try:
                        old_price = product["prices"]["oldPrice"]
                    except:
                        old_price = 0
                    new_product = {
                        "sku": product["sku"],
                        "name": product["name"],
                        "brand": product["brand"],
                        "old_price": old_price,
                        "new_price": product["prices"]["price"],
                        "rating": product["rating"]["average"],
                        "image": product["image"],
                        "url": product["url"]
                    }
                    # print(new_product)
                    products.append(new_product)
                break
        print(len(products))
        
    except Exception as e:
        print(e)
        break

In [85]:
df = pd.DataFrame(products)
df.to_csv('electromenager_products.csv',index=False)

## Get Product Reviews

In [ ]:
for product in products:
    p_sku = product["sku"]
    p_link = f"https://www.jumia.dz/catalog/productratingsreviews/sku/{p_sku}/"
    req = requests.get(p_link)
    s = BeautifulSoup(req.content, "html.parser")
    try:
        num_pages = s.select('a[aria-label="Dernière page"]')[0]["href"].split("=")[1]
    except:
        print("only one page for ", p_link)
    print(num_pages)
    num_reviews = 0
    product_reviews = []

    for page in range(1,int(num_pages)):
        p_link = f"https://www.jumia.dz/catalog/productratingsreviews/sku/{p_sku}/?page={page}"
        req = requests.get(p_link)
        s = BeautifulSoup(req.content, "html.parser")
        for review in s.find_all("article")[1:]:
            # print(review)
            new_review = {
                "reviewer": review.select("span:nth-of-type(2)")[0].text.split("par ")[1],
                "title": review.select("h3")[0].text,
                "review": review.select("p.-pvs")[0].text
            }
            num_reviews += 1
            product_reviews.append(new_review)
    print("num reviews: " ,num_reviews)
    product["reviews"] = product_reviews
    

        

In [131]:
print(products[2])

{'sku': 'SO672HL0Y7ZB8NAFAMZ', 'name': 'Panineuse Mini & Grill A 2 Tranches - Sgt-853 - Noir & Silver', 'brand': 'Sonashi', 'old_price': '4,500 DA', 'new_price': '3,430 DA', 'rating': 4.1, 'image': 'https://dz.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/57/4793/1.jpg?9985', 'url': '/panineuse-mini-grill-a-2-tranches-sgt-853-noir-silver-sonashi-mpg22062.html', 'reviews': [{'reviewer': 'Mehdi', 'title': 'Excellent', 'review': 'Excellent'}, {'reviewer': 'Ouafa', 'title': 'صغيرة بصح هايلة', 'review': 'سهلة التنقل'}, {'reviewer': 'Yazid', 'title': 'un peu déçu par sa taille', 'review': 'Le produit est bon. Mais il est trop petit.'}, {'reviewer': 'Kheira', 'title': 'ارضاءزبنائكم..وانا.من بينهم', 'review': 'الصدق في تعملاتكم شكرا'}, {'reviewer': 'Sif', 'title': 'panineuse sonashi 750w', 'review': 'جيد وسهل الاستعمال وسعر مقبول'}, {'reviewer': 'Nabil', 'title': 'nabiltgr63@gmail.com', 'review': 'صغيرة شويا هدا مكان'}, {'reviewer': 'زهيرة', 'title': 'hassanine.zahira@gmail.com', 

## Saving Data

In [132]:
with open('jumia_electromenager.pickle', 'wb') as f:
    pickle.dump(products, f)

## Loading Data 

In [133]:
with open('jumia_electromenager.pickle', 'rb') as f:
    all_products= pickle.load(f)

In [134]:
with open("electromenager.json", "w") as outfile:
    json.dump(all_products, outfile)